In [1]:
import os

import mne
import mne_bids


In [2]:
def get_all_files(path, suffix, get_bids=False, prefix=None, bids_root=None, verbose=False, extension=None):
    """Return all files in all (sub-)directories of path with given suffixes and prefixes (case-insensitive).

    Args:
        path (string)
        suffix (iterable): e.g. ["vhdr", "edf"] or ".json"
        get_bids (boolean): True if BIDS_Path type should be returned instead of string. Default: False
        bids_root (string/path): Path of BIDS root folder. Only required if get_bids=True.
        prefix (iterable): e.g. ["SelfpacedRota", "ButtonPress] (optional)

    Returns:
        filepaths (list of strings or list of BIDS_Path)
    """

    if isinstance(suffix, str):
        suffix = [suffix]
    if isinstance(prefix, str):
        prefix = [prefix]

    filepaths = []
    for root, dirs, files in os.walk(path):
        for file in files:
            for suff in suffix:
                if file.endswith(suff.lower()):
                    if not prefix:
                        filepaths.append(os.path.join(root, file))
                    else:
                        for pref in prefix:
                            if pref.lower() in file.lower():
                                filepaths.append(os.path.join(root, file))

    bids_paths = filepaths
    if get_bids:
        if not bids_root:
            print("Warning: No root folder given. Please pass bids_root parameter to create a complete BIDS_Path object.")
        bids_paths = []
        for filepath in filepaths:
            entities = mne_bids.get_entities_from_fname(filepath)
            try:
                bids_path = mne_bids.BIDSPath(subject=entities["subject"], session=entities["session"], task=entities["task"], run=entities["run"], acquisition=entities["acquisition"], suffix=entities["suffix"], extension=extension, root=bids_root)
            except ValueError as err:
                print(f"ValueError while creating BIDS_Path object for file {filepath}: {err}")
            else:
                bids_paths.append(bids_path)

    if verbose:
        if not bids_paths:
            print("No corresponding files found.")
        else:    
            print('Corresponding files found:')
            for idx, file in enumerate(bids_paths):
                print(idx, ':', os.path.basename(file))
                
    return bids_paths

In [3]:
root = r'C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata'
files = get_all_files(
    path=root,
    suffix='ieeg.json',
    get_bids=True,
    prefix=None,
    bids_root=root,
    verbose=True,
    extension='json')

Corresponding files found:
0 : sub-001_ses-EphysMedOff01_task-BlockRotationR_acq-StimOffOn_run-01_ieeg.json
1 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.json
2 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_ieeg.json
3 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOn_run-01_ieeg.json
4 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOn_run-02_ieeg.json
5 : sub-001_ses-EphysMedOff01_task-Visuomotor_acq-StimOff_run-01_ieeg.json
6 : sub-001_ses-EphysMedOff01_task-Visuomotor_acq-StimOn_run-01_ieeg.json
7 : sub-001_ses-EphysMedOn01_task-BlockRotationWheel_acq-StimOff_run-01_ieeg.json
8 : sub-001_ses-EphysMedOn01_task-FreeDrawing_acq-StimOff_run-01_ieeg.json
9 : sub-001_ses-EphysMedOn01_task-Rest_acq-StimOff_run-01_ieeg.json
10 : sub-001_ses-EphysMedOn01_task-Rest_acq-StimOn_run-01_ieeg.json
11 : sub-001_ses-EphysMedOn01_task-SelfpacedForceWheel_acq-StimOff_run-01_ieeg.json
12 : sub-001_ses-EphysMedOn02_task-BlockRotationL_acq-StimOffOn_run-01_ieeg.json
13 : sub-00

In [10]:
import json

In [ ]:
for file in files:
    with open(file.fpath) as json_file:
        ieeg_json = json.load(json_file)
    mne_bids.update_sidecar_json(file, {'iEEGElectrodeGroups': str(ieeg_json['iEEGElectrodeGroups'])})

In [16]:
raw_files = get_all_files(
    path=root,
    suffix='vhdr',
    get_bids=True,
    prefix=None,
    bids_root=root,
    verbose=True,
    extension='vhdr')

Corresponding files found:
0 : sub-001_ses-EphysMedOff01_task-BlockRotationR_acq-StimOffOn_run-01_ieeg.vhdr
1 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr
2 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_ieeg.vhdr
3 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOn_run-01_ieeg.vhdr
4 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOn_run-02_ieeg.vhdr
5 : sub-001_ses-EphysMedOff01_task-Visuomotor_acq-StimOff_run-01_ieeg.vhdr
6 : sub-001_ses-EphysMedOff01_task-Visuomotor_acq-StimOn_run-01_ieeg.vhdr
7 : sub-001_ses-EphysMedOn01_task-BlockRotationWheel_acq-StimOff_run-01_ieeg.vhdr
8 : sub-001_ses-EphysMedOn01_task-FreeDrawing_acq-StimOff_run-01_ieeg.vhdr
9 : sub-001_ses-EphysMedOn01_task-Rest_acq-StimOff_run-01_ieeg.vhdr
10 : sub-001_ses-EphysMedOn01_task-Rest_acq-StimOn_run-01_ieeg.vhdr
11 : sub-001_ses-EphysMedOn01_task-SelfpacedForceWheel_acq-StimOff_run-01_ieeg.vhdr
12 : sub-001_ses-EphysMedOn02_task-BlockRotationL_acq-StimOffOn_run-01_ieeg.vhdr
13 : sub-00

# See if all files can still be read

In [ ]:
for file in raw_files:
    mne_bids.read_raw_bids(file, verbose=False)

In [ ]:
tasks = {
    "SelfpacedSpeech": "Selfpaced Speech", 
    "SelfpacedRotationR": "Selfpaced Rotation R",
    "SelfpacedRotationL": "Selfpaced Rotation L",
    "BlockRotationR": "Block Rotation R",
    "BlockRotationL": "Block Rotation L",
    "UPDRSIII": "UPDRS-III",
    "ReadRelaxMoveL": "Read Relax Move L"}

for file in files:
    with open(file.fpath) as json_file:
        ieeg_json = json.load(json_file)
    if ieeg_json['TaskName'] in tasks.keys():
        mne_bids.update_sidecar_json(file, {'TaskName': tasks[ieeg_json['TaskName']]})

### Updated coordsystems from "mni" to "MNI152NLin2009bAsym" using regex tool

### Now check if data still works

In [ ]:
for file in raw_files:
    mne_bids.read_raw_bids(file, verbose=False)

In [29]:
raw = mne_bids.read_raw_bids(raw_files[0], verbose=False)

Extracting parameters from C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-001\ses-EphysMedOff01\ieeg\sub-001_ses-EphysMedOff01_task-BlockRotationR_acq-StimOffOn_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-001\ses-EphysMedOff01\ieeg\sub-001_ses-EphysMedOff01_task-BlockRotationR_acq-StimOffOn_run-01_channels.tsv.
Reading electrode coords from C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-001\ses-EphysMedOff01\ieeg\sub-001_ses-EphysMedOff01_space-mni_electrodes.tsv.


c:\users\richa\github\mne-bids\mne_bids\path.py:1092: RuntimeWarning: Did not find any events.tsv associated with sub-001_ses-EphysMedOff01_task-BlockRotationR_acq-StimOffOn_run-01.

The search_str was "C:\Users\richa\OneDrive - Charité - Universitätsmedizin Berlin\Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-001\**\sub-001_ses-EphysMedOff01*events.tsv"
  warn(msg)
c:\users\richa\github\mne-bids\mne_bids\dig.py:468: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  warn(f"Defaulting coordinate frame to unknown "
c:\users\richa\github\mne-bids\mne_bids\dig.py:519: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage, on_missing='warn', verbose=verbose)
c:\users\richa\github\mne-bids\mne_bids\dig.py:519: RuntimeWarning: DigMontage is only a subset of info. There are 8 channel positions not present in the DigMontage. The required channels are:

['LFP_L_1_2_STN_B', 'LFP_L_1_3_STN_B', '

In [30]:
raw.info

<Info | 10 non-empty values
 bads: []
 ch_names: ANALOG_R_ROTA_C, ECOG_L_1_2_SMC_, ECOG_L_1_3_SMC_, ...
 chs: 1 MISC, 5 ECOG, 8 SEEG
 custom_ref_applied: False
 dig: 5 items (5 EEG)
 highpass: 0.0 Hz
 line_freq: 50
 lowpass: 2500.0 Hz
 meas_date: 2017-10-18 09:31:45 UTC
 nchan: 14
 projs: []
 sfreq: 5000.0 Hz
 subject_info: 25 items (dict)
>

In [31]:
raw.info['dig']

[<DigPoint |     EEG #1 : (-46.0, -15.5, 63.0) mm   : head frame>,
 <DigPoint |     EEG #2 : (-45.5, -5.5, 60.5) mm    : head frame>,
 <DigPoint |     EEG #3 : (-44.5, 4.0, 57.5) mm     : head frame>,
 <DigPoint |     EEG #4 : (-42.0, 14.0, 53.5) mm    : head frame>,
 <DigPoint |     EEG #5 : (-39.0, 22.5, 49.5) mm    : head frame>]

# Now reorder channels.tsv

In [4]:
channel_files = get_all_files(
    path=root,
    suffix='channels.tsv',
    get_bids=False,
    prefix=None,
    bids_root=root,
    verbose=True,
    extension='tsv')

Corresponding files found:
0 : sub-001_ses-EphysMedOff01_task-BlockRotationR_acq-StimOffOn_run-01_channels.tsv
1 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_channels.tsv
2 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_channels.tsv
3 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOn_run-01_channels.tsv
4 : sub-001_ses-EphysMedOff01_task-Rest_acq-StimOn_run-02_channels.tsv
5 : sub-001_ses-EphysMedOff01_task-Visuomotor_acq-StimOff_run-01_channels.tsv
6 : sub-001_ses-EphysMedOff01_task-Visuomotor_acq-StimOn_run-01_channels.tsv
7 : sub-001_ses-EphysMedOn01_task-BlockRotationWheel_acq-StimOff_run-01_channels.tsv
8 : sub-001_ses-EphysMedOn01_task-FreeDrawing_acq-StimOff_run-01_channels.tsv
9 : sub-001_ses-EphysMedOn01_task-Rest_acq-StimOff_run-01_channels.tsv
10 : sub-001_ses-EphysMedOn01_task-Rest_acq-StimOn_run-01_channels.tsv
11 : sub-001_ses-EphysMedOn01_task-SelfpacedForceWheel_acq-StimOff_run-01_channels.tsv
12 : sub-001_ses-EphysMedOn02_task-BlockRotationL_acq-St

In [5]:
import pandas as pd

In [10]:
dfs = list()
for file in channel_files:
    df = pd.read_csv(file, sep='\t', index_col=0)
    dfs.append(df)

In [11]:
dfs[0].columns

Index(['type', 'units', 'low_cutoff', 'high_cutoff', 'description',
       'sampling_frequency', 'status', 'status_description'],
      dtype='object')

In [12]:
dfs[73].columns

Index(['type', 'units', 'low_cutoff', 'high_cutoff', 'reference', 'group',
       'sampling_frequency', 'notch', 'status', 'status_description'],
      dtype='object')

In [53]:
for i, df in enumerate(dfs):
    if 'group' in df.columns:
        df_reorder = df[['type', 'units', 'low_cutoff', 'high_cutoff',
                         'reference', 'group', 'sampling_frequency', 
                         'notch', 'status', 'status_description']]
        df_reorder.to_csv(channel_files[i], sep='\t', na_rep='n/a', index=True)

In [14]:
for i, df in enumerate(dfs):
    if 'group' not in df.columns:
        df['group'] = None
        if 'reference' not in df.columns:
            df['reference'] = None
            if 'notch' not in df.columns:
                df['notch'] = None
        df_reorder = df[['type', 'units', 'low_cutoff', 'high_cutoff',
                         'reference', 'group', 'sampling_frequency', 
                         'notch', 'status', 'status_description']]
        df_reorder.to_csv(channel_files[i], sep='\t', na_rep='n/a', index=True)

In [15]:
dfs_new = list()
for file in channel_files:
    df = pd.read_csv(file, sep='\t', index_col=0)
    dfs_new.append(df)

In [18]:
dfs_new[0]

,type,units,low_cutoff,high_cutoff,reference,group,sampling_frequency,notch,status,status_description
name,,,,,,,,,,
ANALOG_R_ROTA_C,MISC,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
ECOG_L_1_2_SMC_,ECOG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
ECOG_L_1_3_SMC_,ECOG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
ECOG_L_1_4_SMC_,ECOG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
ECOG_L_1_5_SMC_,ECOG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
ECOG_L_1_6_SMC_,ECOG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
LFP_L_1_2_STN_B,SEEG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
LFP_L_1_3_STN_B,SEEG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN
LFP_L_1_4_STN_B,SEEG,µV,0.0,2500.0,NaN,NaN,5000.0,NaN,good,NaN


In [17]:
dfs_new[73]

,type,units,low_cutoff,high_cutoff,reference,group,sampling_frequency,notch,status,status_description
name,,,,,,,,,,
LFP_R_1_STN_MT,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
LFP_R_234_STN_M,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
LFP_R_567_STN_M,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
LFP_R_8_STN_MT,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
LFP_L_1_STN_MT,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,bad,Reference electrode
LFP_L_234_STN_M,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
LFP_L_567_STN_M,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
LFP_L_8_STN_MT,SEEG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
ECOG_L_1_SMC_AT,ECOG,uV,NaN,NaN,NaN,NaN,4000,NaN,good,NaN
